# Data

Loading and appending the Chicago Crime dataset

In [4]:
import pandas as pd
import duckdb
import openai
import time 
import os

Set the path of the CSV files:

In [5]:
path = "./data"

files = [x for x in os.listdir(path = path) if ".csv" in x]

print(files)

['chicago_crime_2022.csv', 'chicago_crime_2023.csv', 'chicago_crime_2021.csv']


In [6]:
chicago_crime = pd.concat((pd.read_csv(path +"/" + f) for f in files), ignore_index=True)

chicago_crime.head

<bound method NDFrame.head of               ID Case Number                    Date                   Block   
0       12589893    JF109865  01/11/2022 03:00:00 PM    087XX S KINGSTON AVE  \
1       12592454    JF113025  01/14/2022 03:55:00 PM       067XX S MORGAN ST   
2       12601676    JF124024  01/13/2022 04:00:00 PM    031XX W AUGUSTA BLVD   
3       12785595    JF346553  08/05/2022 09:00:00 PM  072XX S UNIVERSITY AVE   
4       12808281    JF373517  08/14/2022 02:00:00 PM     055XX W ARDMORE AVE   
...          ...         ...                     ...                     ...   
648826     26461    JE455267  11/24/2021 12:51:00 AM     107XX S LANGLEY AVE   
648827     26041    JE281927  06/28/2021 01:12:00 AM       117XX S LAFLIN ST   
648828     26238    JE353715  08/29/2021 03:07:00 AM    010XX N LAWNDALE AVE   
648829     26479    JE465230  12/03/2021 08:37:00 PM         000XX W 78TH PL   
648830  11138622    JA495186  05/21/2021 12:01:00 AM      019XX N PULASKI RD   

        I

In [7]:
chicago_crime

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,12589893,JF109865,01/11/2022 03:00:00 PM,087XX S KINGSTON AVE,1565,SEX OFFENSE,INDECENT SOLICITATION OF A CHILD,RESIDENCE,False,True,...,7.0,46,17,1194660.0,1847481.0,2022,09/14/2023 03:41:59 PM,41.736409,-87.562410,"(41.736409029, -87.562410309)"
1,12592454,JF113025,01/14/2022 03:55:00 PM,067XX S MORGAN ST,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,RESIDENCE,False,True,...,16.0,68,26,1170805.0,1860170.0,2022,09/14/2023 03:41:59 PM,41.771782,-87.649437,"(41.771782439, -87.649436929)"
2,12601676,JF124024,01/13/2022 04:00:00 PM,031XX W AUGUSTA BLVD,1752,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,RESIDENCE,False,True,...,36.0,23,17,1155171.0,1906486.0,2022,09/14/2023 03:41:59 PM,41.899206,-87.705506,"(41.899206068, -87.705505587)"
3,12785595,JF346553,08/05/2022 09:00:00 PM,072XX S UNIVERSITY AVE,1544,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,APARTMENT,True,False,...,5.0,69,17,1185135.0,1857211.0,2022,09/14/2023 03:41:59 PM,41.763338,-87.597001,"(41.763337967, -87.597001131)"
4,12808281,JF373517,08/14/2022 02:00:00 PM,055XX W ARDMORE AVE,1562,SEX OFFENSE,AGGRAVATED CRIMINAL SEXUAL ABUSE,RESIDENCE,False,False,...,39.0,11,17,1138383.0,1937953.0,2022,09/14/2023 03:41:59 PM,41.985875,-87.766404,"(41.985875279, -87.766403857)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648826,26461,JE455267,11/24/2021 12:51:00 AM,107XX S LANGLEY AVE,0110,HOMICIDE,FIRST DEGREE MURDER,VACANT LOT,False,False,...,9.0,50,01A,1182822.0,1833730.0,2021,09/19/2022 03:41:05 PM,41.698957,-87.606206,"(41.698957409, -87.606205674)"
648827,26041,JE281927,06/28/2021 01:12:00 AM,117XX S LAFLIN ST,0110,HOMICIDE,FIRST DEGREE MURDER,AUTO,False,False,...,34.0,53,01A,1168442.0,1826982.0,2021,09/01/2022 03:42:17 PM,41.680761,-87.659052,"(41.680760863, -87.659051873)"
648828,26238,JE353715,08/29/2021 03:07:00 AM,010XX N LAWNDALE AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,False,False,...,27.0,23,01A,1151525.0,1906643.0,2021,09/19/2022 03:41:05 PM,41.899709,-87.718893,"(41.899709327, -87.718893208)"
648829,26479,JE465230,12/03/2021 08:37:00 PM,000XX W 78TH PL,0110,HOMICIDE,FIRST DEGREE MURDER,PORCH,True,False,...,6.0,69,01A,1177156.0,1852951.0,2021,09/01/2022 03:42:17 PM,41.751832,-87.626374,"(41.751831742, -87.626373808)"


Source: https://platform.openai.com/examples/default-sql-translate

In [8]:
prompt_template = """

Given the following SQL table, your job is to write queries given a user’s request. \n

CREATE TABLE {} ({}) \n

Write a SQL query that returns - {}
"""

In [9]:
def sql_prompt_generator(table_name, col_names, query):
    prompt = prompt_template.format(table_name, col_names, query)
    return prompt


In [19]:
table = "chicago_crime"
col_names = str(list(chicago_crime.columns)).replace('[', '').replace(']', '')
query = "How many cases are from the year 2023?"

p = sql_prompt_generator(table_name = table, col_names = col_names, query = query)

print(p)





Given the following SQL table, your job is to write queries given a user’s request. 


CREATE TABLE chicago_crime ('ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type', 'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude', 'Location') 


Write a SQL query that returns - How many cases are from the year 2023?



In [7]:
openai.api_key = os.getenv('OPENAI_KEY')

In [21]:
openai_api_models = pd.DataFrame(openai.Model.list()["data"])

openai_api_models.head

<bound method NDFrame.head of                                id object     created         owned_by
0     text-search-babbage-doc-001  model  1651172509       openai-dev
1                           gpt-4  model  1687882411           openai
2              curie-search-query  model  1651172509       openai-dev
3                text-davinci-003  model  1669599635  openai-internal
4   text-search-babbage-query-001  model  1651172509       openai-dev
..                            ...    ...         ...              ...
65    gpt-3.5-turbo-instruct-0914  model  1694122472           system
66                       dall-e-2  model  1698798177           system
67                     tts-1-1106  model  1699053241           system
68                  tts-1-hd-1106  model  1699053533           system
69              gpt-3.5-turbo-16k  model  1683758102  openai-internal

[70 rows x 4 columns]>

In [22]:
response = openai.Completion.create(engine = "text-davinci-003",
                                     prompt= p)

In [23]:
query = response["choices"][0]["text"]
print(query)
duckdb.sql(query).show()

SELECT COUNT(*) FROM chicago_crime WHERE `year` = 20


CatalogException: Catalog Error: Scalar Function with name `__postfix does not exist!
Did you mean "!__postfix"?

In [24]:
response = openai.Completion.create(engine = "text-davinci-003",
                                     prompt= p,
                                     n = 2)
print(response)
query = response["choices"][0]["text"]

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8PzR6LpdEHck81Q8PNisYVTdkame9",
  "object": "text_completion",
  "created": 1701205052,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\nSELECT COUNT(*)\nFROM chicago_crime\nWHERE Year",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    },
    {
      "text": "\nSELECT COUNT(*) \nFROM chicago_crime  \n",
      "index": 1,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 134,
    "completion_tokens": 32,
    "total_tokens": 166
  }
}


In [25]:
response = openai.Completion.create(engine = "text-davinci-003",
                                     prompt= p,
                                     n = 2,
                                     temperature = 0)
print(response)
query = response["choices"][0]["text"]

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8PzR6z8MGo0EfUe2we9OYueYNbgLZ",
  "object": "text_completion",
  "created": 1701205052,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\nSELECT COUNT(*) FROM chicago_crime WHERE Year = 2023",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    },
    {
      "text": "\nSELECT COUNT(*) FROM chicago_crime WHERE Year = 2023",
      "index": 1,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 134,
    "completion_tokens": 32,
    "total_tokens": 166
  }
}


In [26]:
response = openai.Completion.create(engine = "text-davinci-003",
                                     prompt= p,
                                     n = 2,
                                     temperature = 2)
print(response)
query = response["choices"][0]["text"]

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8PzR7muUDIG3y2tPuoW7OYPPDyul5",
  "object": "text_completion",
  "created": 1701205053,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "SELECT COUNT(2000) \n sympath vaccination Wife 2014 occupations giving with Policy",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    },
    {
      "text": "\nSELECT COUNT(*) \n    FROM chicago_crime\nWHERE",
      "index": 1,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 134,
    "completion_tokens": 32,
    "total_tokens": 166
  }
}


In [27]:
def lang2sql(api_key, table_name, col_names, query, engine = "text-davinci-003", n = 1, temperature = 0):
    prompt = sql_prompt_generator(table_name = table_name, col_names = col_names, query = query)
    
    openai.api_key = api_key

    response = openai.Completion.create(engine = engine,
                                        prompt= prompt,
                                        n = n,
                                        temperature = temperature)
    return response


In [28]:
table_name = "chicago_crime"
col_names = str(list(chicago_crime.columns)).replace('[', '').replace(']', '')
api_key = os.getenv('OPENAI_KEY')

In [29]:
query = "How many cases are from the year 2023?"

response = lang2sql(api_key = api_key, table_name = table_name, col_names= col_names, query = query)

In [30]:
print(response)
sql_query = response["choices"][0]["text"]
print(sql_query)

duckdb.sql(sql_query).show()

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8PzR8hP1OI5ih0B3tdlPGCNTzu58U",
  "object": "text_completion",
  "created": 1701205054,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\nSELECT COUNT(*) FROM chicago_crime WHERE Year = 2023",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 134,
    "completion_tokens": 16,
    "total_tokens": 150
  }
}

SELECT COUNT(*) FROM chicago_crime WHERE Year = 2023
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       200932 │
└──────────────┘



In [31]:
query = "How many cases ended up with arrest?"

response = lang2sql(api_key = api_key, table_name = table_name, col_names= col_names, query = query)
sql_query = response["choices"][0]["text"]
print(sql_query)


SELECT COUNT(*) 
FROM chicago_crime 



In [32]:
tbl_describe = duckdb.sql("DESCRIBE SELECT * FROM chicago_crime;")
tbl_describe



┌──────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │ column_type │  null   │   key   │ default │  extra  │
│       varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ID                   │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Case Number          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Date                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Block                │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ IUCR                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Primary Type         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Description          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Location Description │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Arrest               │ BOOLEAN     │ YES     │ NUL

In [33]:
col_attr = tbl_describe.df()[["column_name", "column_type"]]

col_attr["column_joint"] = col_attr["column_name"] + " " +  col_attr["column_type"]
col_attr

,column_name,column_type,column_joint
0,ID,BIGINT,ID BIGINT
1,Case Number,VARCHAR,Case Number VARCHAR
2,Date,VARCHAR,Date VARCHAR
3,Block,VARCHAR,Block VARCHAR
4,IUCR,VARCHAR,IUCR VARCHAR
5,Primary Type,VARCHAR,Primary Type VARCHAR
6,Description,VARCHAR,Description VARCHAR
7,Location Description,VARCHAR,Location Description VARCHAR
8,Arrest,BOOLEAN,Arrest BOOLEAN
9,Domestic,BOOLEAN,Domestic BOOLEAN


In [34]:
col_names = str(list(col_attr["column_joint"].values)).replace('[', '').replace(']', '').replace('\'', '')
query = "How many cases ended up with arrest?"
prompt = sql_prompt_generator(table_name = table_name, col_names = col_names, query = query)
print(prompt)



Given the following SQL table, your job is to write queries given a user’s request. 


CREATE TABLE chicago_crime (ID BIGINT, Case Number VARCHAR, Date VARCHAR, Block VARCHAR, IUCR VARCHAR, Primary Type VARCHAR, Description VARCHAR, Location Description VARCHAR, Arrest BOOLEAN, Domestic BOOLEAN, Beat BIGINT, District BIGINT, Ward DOUBLE, Community Area BIGINT, FBI Code VARCHAR, X Coordinate DOUBLE, Y Coordinate DOUBLE, Year BIGINT, Updated On VARCHAR, Latitude DOUBLE, Longitude DOUBLE, Location VARCHAR) 


Write a SQL query that returns - How many cases ended up with arrest?



In [35]:

response = lang2sql(api_key = api_key, table_name = table_name, col_names= col_names, query = query)
print(response)
sql_query = response["choices"][0]["text"]
print(sql_query)

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8PzR9Hrv0Q2BNAgnSHyKiH0P18TO8",
  "object": "text_completion",
  "created": 1701205055,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\nSELECT COUNT(*) \nFROM chicago_crime \n",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 172,
    "completion_tokens": 16,
    "total_tokens": 188
  }
}

SELECT COUNT(*) 
FROM chicago_crime 



In [36]:
system = """
Given the following SQL table, your job is to write queries given a user’s request. 


CREATE TABLE chicago_crime (ID BIGINT, Case Number VARCHAR, Date VARCHAR, Block VARCHAR, IUCR VARCHAR, Primary Type VARCHAR, Description VARCHAR, Location Description VARCHAR, Arrest BOOLEAN, Domestic BOOLEAN, Beat BIGINT, District BIGINT, Ward DOUBLE, Community Area BIGINT, FBI Code VARCHAR, X Coordinate DOUBLE, Y Coordinate DOUBLE, Year BIGINT, Updated On VARCHAR, Latitude DOUBLE, Longitude DOUBLE, Location VARCHAR) 
"""

prompt = "Write a SQL query that returns - How many cases ended up with arrest?"

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": system
    },
    {
      "role": "user",
      "content": prompt
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [37]:
print(response)

{
  "id": "chatcmpl-8PzRA3oj9bRqxq2xP45oL5PXVkvo6",
  "object": "chat.completion",
  "created": 1701205056,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "SELECT COUNT(*) FROM chicago_crime WHERE Arrest = TRUE;"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 134,
    "completion_tokens": 12,
    "total_tokens": 146
  }
}


In [38]:
sql_query = response["choices"][0]["message"]["content"]
print(sql_query)


SELECT COUNT(*) FROM chicago_crime WHERE Arrest = TRUE;


In [39]:
duckdb.sql(sql_query).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        77635 │
└──────────────┘



In [40]:
duckdb.sql("DESCRIBE SELECT * FROM chicago_crime;")

┌──────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │ column_type │  null   │   key   │ default │  extra  │
│       varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ID                   │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Case Number          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Date                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Block                │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ IUCR                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Primary Type         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Description          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Location Description │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Arrest               │ BOOLEAN     │ YES     │ NUL

In [8]:
def create_message(table_name, query):

    class message:
        def __init__(message, system, user, column_names, column_attr):
            message.system = system
            message.user = user
            message.column_names = column_names
            message.column_attr = column_attr

    
    system_template = """

    Given the following SQL table, your job is to write queries given a user’s request. \n

    CREATE TABLE {} ({}) \n
    """

    user_template = "Write a SQL query that returns - {}"
    
    tbl_describe = duckdb.sql("DESCRIBE SELECT * FROM " + table_name +  ";")
    col_attr = tbl_describe.df()[["column_name", "column_type"]]
    col_attr["column_joint"] = col_attr["column_name"] + " " +  col_attr["column_type"]
    col_names = str(list(col_attr["column_joint"].values)).replace('[', '').replace(']', '').replace('\'', '')

    system = system_template.format(table_name, col_names)
    user = user_template.format(query)

    m = message(system = system, user = user, column_names = col_attr["column_name"], column_attr = col_attr["column_type"])
    return m


    

In [12]:
query = "How many cases ended up with arrest?"
msg = create_message(table_name = "chicago_crime", query = query)

print(msg.system)
print(msg.user)
print(msg.column_names)
print(msg.column_attr)



    Given the following SQL table, your job is to write queries given a user’s request. 


    CREATE TABLE chicago_crime (ID BIGINT, Case Number VARCHAR, Date VARCHAR, Block VARCHAR, IUCR VARCHAR, Primary Type VARCHAR, Description VARCHAR, Location Description VARCHAR, Arrest BOOLEAN, Domestic BOOLEAN, Beat BIGINT, District BIGINT, Ward DOUBLE, Community Area BIGINT, FBI Code VARCHAR, X Coordinate DOUBLE, Y Coordinate DOUBLE, Year BIGINT, Updated On VARCHAR, Latitude DOUBLE, Longitude DOUBLE, Location VARCHAR) 

    
Write a SQL query that returns - How many cases ended up with arrest?
0                       ID
1              Case Number
2                     Date
3                    Block
4                     IUCR
5             Primary Type
6              Description
7     Location Description
8                   Arrest
9                 Domestic
10                    Beat
11                District
12                    Ward
13          Community Area
14                FBI Code

In [13]:
query = "How many cases ended up with arrest?"
prompt = create_message(table_name = "chicago_crime", query = query)

In [14]:
  message = [
    {
      "role": "system",
      "content": prompt.system
    },
    {
      "role": "user",
      "content": prompt.user
    }
    ]

message

[{'role': 'system',
  'content': '\n\n    Given the following SQL table, your job is to write queries given a user’s request. \n\n\n    CREATE TABLE chicago_crime (ID BIGINT, Case Number VARCHAR, Date VARCHAR, Block VARCHAR, IUCR VARCHAR, Primary Type VARCHAR, Description VARCHAR, Location Description VARCHAR, Arrest BOOLEAN, Domestic BOOLEAN, Beat BIGINT, District BIGINT, Ward DOUBLE, Community Area BIGINT, FBI Code VARCHAR, X Coordinate DOUBLE, Y Coordinate DOUBLE, Year BIGINT, Updated On VARCHAR, Latitude DOUBLE, Longitude DOUBLE, Location VARCHAR) \n\n    '},
 {'role': 'user',
  'content': 'Write a SQL query that returns - How many cases ended up with arrest?'}]

In [15]:
 response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = message,
        temperature = 0,
        max_tokens = 256,
        frequency_penalty = 0,
        presence_penalty = 0)

In [16]:
response

<OpenAIObject chat.completion id=chatcmpl-8QPyBs2f1ZuG1IRxF9pVBQNddX4Vl at 0xffff788f53a0> JSON: {
  "id": "chatcmpl-8QPyBs2f1ZuG1IRxF9pVBQNddX4Vl",
  "object": "chat.completion",
  "created": 1701307047,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "SELECT COUNT(*) FROM chicago_crime WHERE Arrest = true;"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 137,
    "completion_tokens": 12,
    "total_tokens": 149
  },
  "system_fingerprint": null
}

In [24]:
sql = response["choices"][0]["message"]["content"]

print(sql)

SELECT COUNT(*) FROM chicago_crime WHERE Arrest = true;


In [25]:
duckdb.sql(sql).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        77635 │
└──────────────┘



In [9]:
def add_quotes(query, col_names):
    for i in col_names:
        if i in query:
            l = query.find(i)
            if query[l-1] != "'" and query[l-1] != '"': 
                query = str(query).replace(i, '"' + i + '"') 

    return(query)

In [59]:
q1 ="SELECT COUNT(*) FROM chicago_crime WHERE Primary Type = 'ROBBERY';"
q2 ="SELECT COUNT(*) FROM chicago_crime WHERE \"Primary Type\" = 'ROBBERY';"
q3 ="SELECT COUNT(*) FROM chicago_crime WHERE 'Primary Type' = 'ROBBERY';"

print(add_quotes(query = q1, col_names= prompt.column_names))
print(add_quotes(query = q2, col_names= prompt.column_names))
print(add_quotes(query = q3, col_names= prompt.column_names))

SELECT COUNT(*) FROM chicago_crime WHERE "Primary Type" = 'ROBBERY';
SELECT COUNT(*) FROM chicago_crime WHERE "Primary Type" = 'ROBBERY';
SELECT COUNT(*) FROM chicago_crime WHERE 'Primary Type' = 'ROBBERY';


In [10]:
add_quotes(query = sql, col_names = prompt.column_names)

NameError: name 'sql' is not defined

In [11]:
def lang2sql(api_key, table_name, query, model = "gpt-3.5-turbo", temperature = 0, max_tokens = 256, frequency_penalty = 0,presence_penalty= 0):
    class response:
        def __init__(output, message, response, sql):
            output.message = message
            output.response = response
            output.sql = sql

    openai.api_key = api_key

    m = create_message(table_name = table_name, query = query)

    message = [
    {
      "role": "system",
      "content": m.system
    },
    {
      "role": "user",
      "content": m.user
    }
    ]
    
    openai_response = openai.ChatCompletion.create(
        model = model,
        messages = message,
        temperature = temperature,
        max_tokens = max_tokens,
        frequency_penalty = frequency_penalty,
        presence_penalty = presence_penalty)
    
    sql_query = add_quotes(query = openai_response["choices"][0]["message"]["content"], col_names = m.column_names)

    output = response(message = m, response = openai_response, sql = sql_query)

    return output


In [13]:
api_key = os.getenv('OPENAI_KEY')

In [62]:
query = "How many cases ended up with arrest?"
response = lang2sql(api_key = api_key, table_name = "chicago_crime", query = query)

In [63]:
print(response.message)
print(response.sql)
print(response.response)



<__main__.create_message.<locals>.message object at 0xffff6fe96920>
SELECT COUNT(*) FROM chicago_crime WHERE "Arrest" = true;
{
  "id": "chatcmpl-8QlD1rolEAv7sPtagPTiNjVGfxewS",
  "object": "chat.completion",
  "created": 1701388691,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "SELECT COUNT(*) FROM chicago_crime WHERE Arrest = true;"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 137,
    "completion_tokens": 12,
    "total_tokens": 149
  },
  "system_fingerprint": null
}


In [64]:
duckdb.sql(response.sql).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        77635 │
└──────────────┘



In [65]:
duckdb.sql(response.sql).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        77635 │
└──────────────┘



In [37]:
query = "How many cases ended up with arrest during 2022"
response = lang2sql(api_key = api_key, table_name = "chicago_crime", query = query)
print(response.sql)

SELECT COUNT(*) FROM chicago_crime WHERE "Arrest" = TRUE AND "Year" = 2022;


In [38]:
duckdb.sql(response.sql).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        27805 │
└──────────────┘



In [152]:


query = "How many cases ended up with arrest between 2022 and 2023?"
response = lang2sql(api_key = api_key, table_name = "chicago_crime", query = query)
duckdb.sql(response.sql).show()

SELECT COUNT(*) FROM chicago_crime WHERE "Arrest" = TRUE AND "Year" BETWEEN 2022 AND 2023;
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        51265 │
└──────────────┘



In [39]:
query = "Summarize the cases by primary type"
response = lang2sql(api_key = api_key, table_name = "chicago_crime", query = query)

print(response.sql)

duckdb.sql(response.sql).show()

SELECT "Primary Type", COUNT(*) as TotalCases
FROM chicago_crime
GROUP BY "Primary Type"
┌───────────────────────────────────┬────────────┐
│           Primary Type            │ TotalCases │
│              varchar              │   int64    │
├───────────────────────────────────┼────────────┤
│ MOTOR VEHICLE THEFT               │      54934 │
│ ROBBERY                           │      25082 │
│ WEAPONS VIOLATION                 │      24672 │
│ INTERFERENCE WITH PUBLIC OFFICER  │       1161 │
│ OBSCENITY                         │        127 │
│ STALKING                          │       1206 │
│ BATTERY                           │     115760 │
│ OFFENSE INVOLVING CHILDREN        │       5177 │
│ CRIMINAL TRESPASS                 │      11255 │
│ PUBLIC PEACE VIOLATION            │       1980 │
│    ·                              │         ·  │
│    ·                              │         ·  │
│    ·                              │         ·  │
│ ASSAULT                           │      5

In [15]:
query = "How many cases is the type of robbery?"
response = lang2sql(api_key = api_key, table_name = "chicago_crime", query = query)

print(response.sql)

duckdb.sql(response.sql).show()

SELECT COUNT(*) FROM chicago_crime WHERE "Primary Type" = 'ROBBERY';
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        25082 │
└──────────────┘



In [67]:
duckdb.sql(response.sql).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        25082 │
└──────────────┘

